# Public Commenting in a Pandemic

## Analysis

In [4]:
# import packages
import pandas as pd
import numpy as np
import json
import time
import os
import datetime
import re

In [5]:
# Specify the path of the folder where the data are saved
filePath = "C:/Users/mark/Box Sync/_MF/Assignments/Insights/Public Commenting and COVID-19/Data/Annual/"

## 2020

In [6]:
# load CSV
fileName = 'cleaned_PS_2020.csv'

with open(filePath+fileName,'r',encoding='utf-8') as loadfile:
    df2020 = pd.read_csv(loadfile, index_col='index')
df2020.info()

C:\Users\mark\anaconda3\envs\pipenv1\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\mark\anaconda3\envs\pipenv1\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1368244 entries, 0 to 1368243
Data columns (total 18 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   agencyBranch     1368244 non-null  object
 1   agencyParent     1368244 non-null  object
 2   agencyAcronym    1368244 non-null  object
 3   docketId         1368244 non-null  object
 4   docketType       1368244 non-null  object
 5   documentId       1368244 non-null  object
 6   submitterName    1269579 non-null  object
 7   title            1368244 non-null  object
 8   organization     84303 non-null    object
 9   attachmentCount  1368244 non-null  int64 
 10  analysisDate     1368244 non-null  object
 11  analysisYear     1368244 non-null  int64 
 12  analysisMonth    1368244 non-null  int64 
 13  commentsWithMCC  1368244 non-null  int64 
 14  commentsNoMCC    1368244 non-null  int64 
 15  MCCfilter        1368244 non-null  object
 16  MCCnumber        1368244 non-null  i

index
0          Unidentified
1          Unidentified
2          Unidentified
3          Unidentified
4          Unidentified
               ...     
1368239              No
1368240             Yes
1368241              No
1368242              No
1368243              No
Name: represent, Length: 1368244, dtype: object